# SQL Layout Analysis Playground (TPC-H)

这个 notebook 用于做两件事：
1. 统计多条 SQL 中各表/各列在 `WHERE` / `JOIN ON` 中的使用情况（含谓词类型、layout heuristic score）
2. 结合 `workloads/stats/*.yaml` 的列分布信息（`cardinality/kind/topk/hist`）做分析与画图

说明：当前环境没有 `sqlglot`，工具会使用一个面向 TPC-H 的启发式解析器（够你先 play 和 debug）。以后装上 `sqlglot` 可以继续增强精度。

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')

ROOT = Path.cwd()
if not (ROOT / 'notebooks').exists() and (ROOT.parent / 'notebooks').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from notebooks import sql_layout_toolkit as slt

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
ROOT

In [ ]:
# ---- Config (edit me) ----
# Option A: use the RQ6 TPCH-all workload stream(s)
SQL_FILES = slt.example_tpch_file_selection(stream='stream_1', root=ROOT / 'workloads/rq6_tpch_all')

# Option B: custom glob examples
# SQL_FILES = slt.discover_sql_files(include_globs=['workloads/rq6_tpch_all/stream_*/query_*.sql'])
# SQL_FILES = slt.discover_sql_files(include_globs=['workloads/tpch_16_Q*/**/*.sql'])

STATS_YAML = ROOT / 'workloads/stats/tpch_1_stats.yaml'
TOP_K = 12
TABLES_TO_PLOT = ['lineitem', 'orders']

len(SQL_FILES), SQL_FILES[:3]

In [ ]:
result = slt.analyze_sql_and_stats(SQL_FILES, STATS_YAML)
analysis = result['analysis']
predicates = result['predicates']
summary = result['summary']
summary_enriched = result['summary_enriched']
cooccurrence = result['cooccurrence']
join_summary = result['join_summary']
stats_df = result['stats']

analysis.meta

In [ ]:
print('Predicate events:', len(predicates))
print('Columns in summary:', len(summary))
print('Tables found:', sorted(summary['table'].dropna().unique().tolist()))

display(slt.summarize_tables_for_display(summary_enriched, top_k=TOP_K))

In [ ]:
# 你最关心的结果 1：每个表哪些列被用到了 + 分布情况如何
cols = [
    'table', 'column', 'score', 'occurrences', 'queries', 'filters', 'joins',
    'eq', 'in', 'range', 'like', 'null',
    'kind', 'cardinality', 'count', 'unique_ratio', 'distinctness_bucket', 'is_low_cardinality', 'is_unique_like'
]
display(summary_enriched[cols].sort_values(['table', 'score'], ascending=[True, False]))

In [ ]:
# 按表看 join key 统计（单独看，方便决定是否要把 join key 纳入 layout）
display(join_summary.sort_values(['join_occurrences', 'join_queries'], ascending=False).head(50))

In [ ]:
# 你最关心的结果 2：画图（按表）
for tbl in TABLES_TO_PLOT:
    if tbl not in set(summary_enriched['table']):
        print(f'skip {tbl}: not found')
        continue
    slt.plot_top_columns(summary_enriched, table=tbl, top_k=TOP_K)
    plt.show()
    slt.plot_predicate_mix(summary_enriched, table=tbl, top_k=min(TOP_K, 10))
    plt.show()


In [ ]:
# score vs distinctness (cardinality/count)
# 对于很低基数列（类别列）和近唯一列（key），这张图很直观
slt.plot_score_vs_distinctness(summary_enriched, table=None)
plt.show()

In [ ]:
# 多维 layout 候选：看列共现（只统计 filter 列共现）
for tbl in TABLES_TO_PLOT:
    g = cooccurrence[cooccurrence['table'] == tbl] if not cooccurrence.empty else cooccurrence
    if g.empty:
        print(f'No cooccurrence for {tbl}')
        continue
    display(g.sort_values('count', ascending=False).head(20))
    try:
        slt.plot_cooccurrence_heatmap(cooccurrence, table=tbl, min_count=1)
        plt.show()
    except Exception as e:
        print(f'heatmap skipped for {tbl}: {e}')


In [ ]:
# 可选：自定义权重（例如更强调 range，或者取消 lineitem boost）
custom_pred_weight = dict(slt.PRED_WEIGHT_DEFAULT)
custom_pred_weight['range'] = 4.0
custom_table_weight = {}  # e.g. {'lineitem': 1.0}

result_custom = slt.analyze_sql_and_stats(SQL_FILES, STATS_YAML, pred_weight=custom_pred_weight, table_role_weight=custom_table_weight)
display(slt.summarize_tables_for_display(result_custom['summary_enriched'], top_k=8))